In [ ]:
from redpitaya import *
from math import pi, sin
from time import sleep
import matplotlib.pyplot as plt

rp = redpitaya()

channels = (0, 1)

buf_size = rp.AcqGetBufSize()

time = [(2*pi)/buf_size*i for i in range(buf_size)]
x = [
    [sin(t) + ((1.0/3.0) * sin(t * 3))           for t in time],
    [(1.0/2.0) * sin(t) + (1.0/4.0) * sin(t * 4) for t in time]
]

amplitude = (0.7, 1.0)
frequency = (4000.0, 4000.0)
    
for ch in channels:
    rp.GenWaveform   (ch, rp.WAVEFORM_ARBITRARY)
    rp.GenArbWaveform(ch, x[ch]);
    rp.GenAmp        (ch, amplitude[ch]);
    rp.GenFreq       (ch, frequency[ch]);
    rp.GenOutEnable  (ch);

In [ ]:
rp.AcqReset()
rp.AcqSetDecimation(1)
rp.AcqSetTriggerLevel(rp.CH_1, 0.1)
rp.AcqSetTriggerDelay(rp.CH_1)

rp.AcqStart()
sleep(0.1)
rp.AcqSetTriggerSrc(rp.TRIG_SRC_CHA_PE)
while rp.AcqGetTriggerState() == rp.TRIG_STATE_WAITING:
    pass
print('triggered')

size = rp.AcqGetBufSize()
buff = [rp.AcqGetOldestDataV(i, size) for i in (rp.CH_1, rp.CH_2)]
#plot(buff1)
#plot(buff2)

In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d
from bokeh.plotting import figure
output_notebook()

N = size
x = np.linspace(0,N,N)

colors = ('red', 'blue')
hover = HoverTool(mode = 'vline', tooltips=[("T", "@x"), ("V", "@y")])
tools = "pan,wheel_zoom,box_zoom,reset,crosshair"
p = figure(plot_height=400, plot_width=900, title="oscilloscope", toolbar_location="above", tools=(tools, hover))
p.x_range=Range1d(0, N)
p.y_range=Range1d(-1.1, 1.1)
r = [p.line(x, buff[i], line_width=1, line_alpha=0.7, color=colors[i]) for i in channels]

# get and explicit handle to update the next show cell with
target = show(p, notebook_handle=True)